<a href="https://colab.research.google.com/github/rithikkulkarni/Video-Virality/blob/main/notebooks/feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Download thumbnails and extract simple visual features


In [ ]:
# Load the CSV
import pandas as pd

url = "https://raw.githubusercontent.com/rithikkulkarni/Video-Virality/refs/heads/main/data/video_details.csv"
df = pd.read_csv(url)
df.head()

,kind,etag,id,channel_id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,snippet.categoryId,snippet.liveBroadcastContent,snippet.defaultLanguage,snippet.localized.title,snippet.localized.description,snippet.defaultAudioLanguage,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount
0,youtube#video,UxEp6XJfLMB4Fz5eWVq36fPYfTc,HbyW-LwzXuM,UC0lG3Ihe4LGV851lODRIS5g,2025-06-20T13:45:02Z,UC0lG3Ihe4LGV851lODRIS5g,A one-pot sautéed chicken dish full of old-sch...,A great way to practice your sauté skills with...,https://i.ytimg.com/vi/HbyW-LwzXuM/default.jpg,120,...,26,none,en,A one-pot sautéed chicken dish full of old-sch...,A great way to practice your sauté skills with...,en,9848.0,899.0,0,46.0
1,youtube#video,DZqEe-E-5XcWRJEjZZj-1v021XE,y9RfNX5RRx8,UC0lG3Ihe4LGV851lODRIS5g,2025-06-09T08:25:09Z,UC0lG3Ihe4LGV851lODRIS5g,How to Learn French Techniques and Recipes,Master the French sauté technique with this si...,https://i.ytimg.com/vi/y9RfNX5RRx8/default.jpg,120,...,26,none,en,How to Learn French Techniques and Recipes,Master the French sauté technique with this si...,en,17134.0,1013.0,0,45.0
2,youtube#video,Ty1cwmDXz55VVEUXhtPpWAvYvmQ,5y8P_ff-ZR0,UC0lG3Ihe4LGV851lODRIS5g,2025-06-04T10:24:47Z,UC0lG3Ihe4LGV851lODRIS5g,The One Book Every Serious Cook Should Own,"If you’re serious about cooking, the Larousse ...",https://i.ytimg.com/vi/5y8P_ff-ZR0/default.jpg,120,...,26,none,en,The One Book Every Serious Cook Should Own,"If you’re serious about cooking, the Larousse ...",en,9450.0,558.0,0,68.0
3,youtube#video,Ji2fd_6HSZrZKLubCJeH6Z5Lk7E,1rMmAp2RZhg,UC0lG3Ihe4LGV851lODRIS5g,2025-05-28T10:39:28Z,UC0lG3Ihe4LGV851lODRIS5g,Salted Caramel Ice Cream – A Taste of Our New ...,Our latest course has arrived—just in time for...,https://i.ytimg.com/vi/1rMmAp2RZhg/default.jpg,120,...,26,none,en,Salted Caramel Ice Cream – A Taste of Our New ...,Our latest course has arrived—just in time for...,en,10097.0,633.0,0,35.0
4,youtube#video,cmA8BDkG4mY6BmkpcVrcayicejw,RpBn_deP_ro,UC0lG3Ihe4LGV851lODRIS5g,2025-05-13T11:16:53Z,UC0lG3Ihe4LGV851lODRIS5g,Learn to make French style ice creams 😉,Available now 🍧 Get the Early bird price disco...,https://i.ytimg.com/vi/RpBn_deP_ro/default.jpg,120,...,26,none,en,Learn to make French style ice creams 😉,Available now 🍧 Get the Early bird price disco...,en,7456.0,183.0,0,2.0


In [ ]:
# Extract video IDs
video_ids = df["id"].dropna().unique().tolist()
print(f"Found {len(video_ids)} video IDs.")

Found 15769 video IDs.


In [ ]:
# Download thumbnails
import os
import requests
from PIL import Image
from io import BytesIO

os.makedirs("thumbnails", exist_ok=True)

def download_thumbnail(video_id):
    url = f"https://i.ytimg.com/vi/{video_id}/mqdefault.jpg"
    path = f"thumbnails/{video_id}.jpg"
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content)).convert('RGB')
        img.save(path)
        return path
    except Exception as e:
        print(f"Error downloading {video_id}: {e}")
        return None

paths = [download_thumbnail(vid) for vid in video_ids]


In [ ]:
# Extract visual features
import numpy as np

def extract_features(img_path, video_id):
    try:
        img = Image.open(img_path).convert('RGB')
        img_array = np.array(img)

        avg_color = np.mean(img_array, axis=(0, 1))
        brightness = np.mean(img_array)
        contrast = np.std(img_array)

        return {
            "video_id": video_id,
            "avg_red": avg_color[0],
            "avg_green": avg_color[1],
            "avg_blue": avg_color[2],
            "brightness": brightness,
            "contrast": contrast
        }
    except Exception as e:
        print(f"Failed to extract for {video_id}: {e}")
        return None

feature_data = [extract_features(f"thumbnails/{vid}.jpg", vid) for vid in video_ids]
features_df = pd.DataFrame([f for f in feature_data if f is not None])
features_df.head()


,video_id,avg_red,avg_green,avg_blue,brightness,contrast
0,HbyW-LwzXuM,157.403733,119.404566,96.229705,124.346001,77.101002
1,y9RfNX5RRx8,110.827431,87.259444,71.066285,89.717720,60.068041
2,5y8P_ff-ZR0,110.985295,124.879844,138.156493,124.673877,86.164616
3,1rMmAp2RZhg,160.647205,141.590087,131.704167,144.647153,43.450358
4,RpBn_deP_ro,103.049583,59.770451,60.072240,74.297425,64.093231


In [ ]:
# Merge features_df with original df to created final "merged_df"

# Merge image features with original DataFrame
merged_df = pd.merge(df, features_df, left_on='id', right_on='video_id', how='left')

# Drop the redundant 'video_id' column
merged_df = merged_df.drop('video_id', axis=1)

In [ ]:
merged_df.head()

,kind,etag,id,channel_id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,snippet.defaultAudioLanguage,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount,avg_red,avg_green,avg_blue,brightness,contrast
0,youtube#video,UxEp6XJfLMB4Fz5eWVq36fPYfTc,HbyW-LwzXuM,UC0lG3Ihe4LGV851lODRIS5g,2025-06-20T13:45:02Z,UC0lG3Ihe4LGV851lODRIS5g,A one-pot sautéed chicken dish full of old-sch...,A great way to practice your sauté skills with...,https://i.ytimg.com/vi/HbyW-LwzXuM/default.jpg,120,...,en,9848.0,899.0,0,46.0,157.403733,119.404566,96.229705,124.346001,77.101002
1,youtube#video,DZqEe-E-5XcWRJEjZZj-1v021XE,y9RfNX5RRx8,UC0lG3Ihe4LGV851lODRIS5g,2025-06-09T08:25:09Z,UC0lG3Ihe4LGV851lODRIS5g,How to Learn French Techniques and Recipes,Master the French sauté technique with this si...,https://i.ytimg.com/vi/y9RfNX5RRx8/default.jpg,120,...,en,17134.0,1013.0,0,45.0,110.827431,87.259444,71.066285,89.717720,60.068041
2,youtube#video,Ty1cwmDXz55VVEUXhtPpWAvYvmQ,5y8P_ff-ZR0,UC0lG3Ihe4LGV851lODRIS5g,2025-06-04T10:24:47Z,UC0lG3Ihe4LGV851lODRIS5g,The One Book Every Serious Cook Should Own,"If you’re serious about cooking, the Larousse ...",https://i.ytimg.com/vi/5y8P_ff-ZR0/default.jpg,120,...,en,9450.0,558.0,0,68.0,110.985295,124.879844,138.156493,124.673877,86.164616
3,youtube#video,Ji2fd_6HSZrZKLubCJeH6Z5Lk7E,1rMmAp2RZhg,UC0lG3Ihe4LGV851lODRIS5g,2025-05-28T10:39:28Z,UC0lG3Ihe4LGV851lODRIS5g,Salted Caramel Ice Cream – A Taste of Our New ...,Our latest course has arrived—just in time for...,https://i.ytimg.com/vi/1rMmAp2RZhg/default.jpg,120,...,en,10097.0,633.0,0,35.0,160.647205,141.590087,131.704167,144.647153,43.450358
4,youtube#video,cmA8BDkG4mY6BmkpcVrcayicejw,RpBn_deP_ro,UC0lG3Ihe4LGV851lODRIS5g,2025-05-13T11:16:53Z,UC0lG3Ihe4LGV851lODRIS5g,Learn to make French style ice creams 😉,Available now 🍧 Get the Early bird price disco...,https://i.ytimg.com/vi/RpBn_deP_ro/default.jpg,120,...,en,7456.0,183.0,0,2.0,103.049583,59.770451,60.072240,74.297425,64.093231


In [ ]:
# Export features to CSV
merged_df.to_csv("video_details_v2.csv", index=False)
print("Image features exported to image_features.csv")

Image features exported to image_features.csv


# Step 2: Text-based feature extraction

In [ ]:
# Load the CSV
import pandas as pd

url = "https://raw.githubusercontent.com/rithikkulkarni/Video-Virality/refs/heads/main/data/video_details_v2.csv"
df = pd.read_csv(url)
df.head()

,kind,etag,id,channel_id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,snippet.defaultAudioLanguage,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount,avg_red,avg_green,avg_blue,brightness,contrast
0,youtube#video,UxEp6XJfLMB4Fz5eWVq36fPYfTc,HbyW-LwzXuM,UC0lG3Ihe4LGV851lODRIS5g,2025-06-20T13:45:02Z,UC0lG3Ihe4LGV851lODRIS5g,A one-pot sautéed chicken dish full of old-sch...,A great way to practice your sauté skills with...,https://i.ytimg.com/vi/HbyW-LwzXuM/default.jpg,120,...,en,9848.0,899.0,0,46.0,157.403733,119.404566,96.229705,124.346001,77.101002
1,youtube#video,DZqEe-E-5XcWRJEjZZj-1v021XE,y9RfNX5RRx8,UC0lG3Ihe4LGV851lODRIS5g,2025-06-09T08:25:09Z,UC0lG3Ihe4LGV851lODRIS5g,How to Learn French Techniques and Recipes,Master the French sauté technique with this si...,https://i.ytimg.com/vi/y9RfNX5RRx8/default.jpg,120,...,en,17134.0,1013.0,0,45.0,110.827431,87.259444,71.066285,89.717720,60.068041
2,youtube#video,Ty1cwmDXz55VVEUXhtPpWAvYvmQ,5y8P_ff-ZR0,UC0lG3Ihe4LGV851lODRIS5g,2025-06-04T10:24:47Z,UC0lG3Ihe4LGV851lODRIS5g,The One Book Every Serious Cook Should Own,"If you’re serious about cooking, the Larousse ...",https://i.ytimg.com/vi/5y8P_ff-ZR0/default.jpg,120,...,en,9450.0,558.0,0,68.0,110.985295,124.879844,138.156493,124.673877,86.164616
3,youtube#video,Ji2fd_6HSZrZKLubCJeH6Z5Lk7E,1rMmAp2RZhg,UC0lG3Ihe4LGV851lODRIS5g,2025-05-28T10:39:28Z,UC0lG3Ihe4LGV851lODRIS5g,Salted Caramel Ice Cream – A Taste of Our New ...,Our latest course has arrived—just in time for...,https://i.ytimg.com/vi/1rMmAp2RZhg/default.jpg,120,...,en,10097.0,633.0,0,35.0,160.647205,141.590087,131.704167,144.647153,43.450358
4,youtube#video,cmA8BDkG4mY6BmkpcVrcayicejw,RpBn_deP_ro,UC0lG3Ihe4LGV851lODRIS5g,2025-05-13T11:16:53Z,UC0lG3Ihe4LGV851lODRIS5g,Learn to make French style ice creams 😉,Available now 🍧 Get the Early bird price disco...,https://i.ytimg.com/vi/RpBn_deP_ro/default.jpg,120,...,en,7456.0,183.0,0,2.0,103.049583,59.770451,60.072240,74.297425,64.093231


In [ ]:
from textblob import TextBlob
import re

# Define sets of keywords
clickbait_words = {
    "amazing", "shocking", "unbelievable", "top", "ultimate", "must",
    "insane", "you won’t believe", "secret", "revealed", "hack"
}
power_words = {
    "best", "new", "crazy", "instant", "guaranteed", "proven", "genius"
}
timed_words = {
    "now", "today", "this week", "this month", "this year", "2024", "2025"
}

def extract_title_features(title):
    blob = TextBlob(title)
    words = title.split()
    punctuation = set("!?.,:;-()[]{}")

    upper_words = [w for w in words if w.isupper()]
    letters = re.findall(r'[A-Za-z]', title)
    uppercase_letters = [c for c in letters if c.isupper()]

    clickbait_score = sum(word.lower() in clickbait_words for word in words)
    power_word_hit = any(word.lower() in power_words for word in words)
    timed_word_hit = any(word.lower() in timed_words for word in words)

    return {
        "title_sentiment": blob.sentiment.polarity,
        "title_subjectivity": blob.sentiment.subjectivity,
        "has_question": int("?" in title),
        "has_exclamation": int("!" in title),
        "starts_with_keyword": int(words[0].lower() in {"how", "why", "what", "when", "where", "who"} if words else 0),
        "title_length": len(title),
        "word_count": len(words),
        "punctuation_count": sum(1 for c in title if c in punctuation),
        "uppercase_word_count": len(upper_words),
        "percent_letters_uppercase": round(len(uppercase_letters) / len(letters), 3) if letters else 0,
        "has_numbers": int(bool(re.search(r"\d", title))),
        "clickbait_score": clickbait_score,
        "has_power_words": int(power_word_hit),
        "has_timed_words": int(timed_word_hit)
    }

In [ ]:
# Load the CSV that has titles in it
import pandas as pd

url = "https://raw.githubusercontent.com/rithikkulkarni/Video-Virality/refs/heads/main/data/video_details.csv"
title_df = pd.read_csv(url)

# Extract only the title and video id from the original csv so we can join it to df by video id to add the title column
title_df = title_df[['snippet.title', 'id', 'channel_id', 'statistics.viewCount', 'statistics.likeCount', 'statistics.commentCount']]
title_df = title_df.rename(columns={'snippet.title': 'title', 'id': 'video_id', 'statistics.viewCount': 'viewCount', 'statistics.likeCount': 'likeCount', 'statistics.commentCount': 'commentCount'})
title_df.head()

,title,video_id,channel_id,viewCount,likeCount,commentCount
0,A one-pot sautéed chicken dish full of old-sch...,HbyW-LwzXuM,UC0lG3Ihe4LGV851lODRIS5g,9848.0,899.0,46.0
1,How to Learn French Techniques and Recipes,y9RfNX5RRx8,UC0lG3Ihe4LGV851lODRIS5g,17134.0,1013.0,45.0
2,The One Book Every Serious Cook Should Own,5y8P_ff-ZR0,UC0lG3Ihe4LGV851lODRIS5g,9450.0,558.0,68.0
3,Salted Caramel Ice Cream – A Taste of Our New ...,1rMmAp2RZhg,UC0lG3Ihe4LGV851lODRIS5g,10097.0,633.0,35.0
4,Learn to make French style ice creams 😉,RpBn_deP_ro,UC0lG3Ihe4LGV851lODRIS5g,7456.0,183.0,2.0


In [ ]:
# Merge to add title column
df = pd.merge(df, title_df, left_on='id', right_on='video_id', how='left')
df.head()

,kind,etag,id,channel_id_x,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,avg_green,avg_blue,brightness,contrast,title,video_id,channel_id_y,viewCount,likeCount,commentCount
0,youtube#video,UxEp6XJfLMB4Fz5eWVq36fPYfTc,HbyW-LwzXuM,UC0lG3Ihe4LGV851lODRIS5g,2025-06-20T13:45:02Z,UC0lG3Ihe4LGV851lODRIS5g,A one-pot sautéed chicken dish full of old-sch...,A great way to practice your sauté skills with...,https://i.ytimg.com/vi/HbyW-LwzXuM/default.jpg,120,...,119.404566,96.229705,124.346001,77.101002,A one-pot sautéed chicken dish full of old-sch...,HbyW-LwzXuM,UC0lG3Ihe4LGV851lODRIS5g,9848.0,899.0,46.0
1,youtube#video,DZqEe-E-5XcWRJEjZZj-1v021XE,y9RfNX5RRx8,UC0lG3Ihe4LGV851lODRIS5g,2025-06-09T08:25:09Z,UC0lG3Ihe4LGV851lODRIS5g,How to Learn French Techniques and Recipes,Master the French sauté technique with this si...,https://i.ytimg.com/vi/y9RfNX5RRx8/default.jpg,120,...,87.259444,71.066285,89.717720,60.068041,How to Learn French Techniques and Recipes,y9RfNX5RRx8,UC0lG3Ihe4LGV851lODRIS5g,17134.0,1013.0,45.0
2,youtube#video,Ty1cwmDXz55VVEUXhtPpWAvYvmQ,5y8P_ff-ZR0,UC0lG3Ihe4LGV851lODRIS5g,2025-06-04T10:24:47Z,UC0lG3Ihe4LGV851lODRIS5g,The One Book Every Serious Cook Should Own,"If you’re serious about cooking, the Larousse ...",https://i.ytimg.com/vi/5y8P_ff-ZR0/default.jpg,120,...,124.879844,138.156493,124.673877,86.164616,The One Book Every Serious Cook Should Own,5y8P_ff-ZR0,UC0lG3Ihe4LGV851lODRIS5g,9450.0,558.0,68.0
3,youtube#video,Ji2fd_6HSZrZKLubCJeH6Z5Lk7E,1rMmAp2RZhg,UC0lG3Ihe4LGV851lODRIS5g,2025-05-28T10:39:28Z,UC0lG3Ihe4LGV851lODRIS5g,Salted Caramel Ice Cream – A Taste of Our New ...,Our latest course has arrived—just in time for...,https://i.ytimg.com/vi/1rMmAp2RZhg/default.jpg,120,...,141.590087,131.704167,144.647153,43.450358,Salted Caramel Ice Cream – A Taste of Our New ...,1rMmAp2RZhg,UC0lG3Ihe4LGV851lODRIS5g,10097.0,633.0,35.0
4,youtube#video,cmA8BDkG4mY6BmkpcVrcayicejw,RpBn_deP_ro,UC0lG3Ihe4LGV851lODRIS5g,2025-05-13T11:16:53Z,UC0lG3Ihe4LGV851lODRIS5g,Learn to make French style ice creams 😉,Available now 🍧 Get the Early bird price disco...,https://i.ytimg.com/vi/RpBn_deP_ro/default.jpg,120,...,59.770451,60.072240,74.297425,64.093231,Learn to make French style ice creams 😉,RpBn_deP_ro,UC0lG3Ihe4LGV851lODRIS5g,7456.0,183.0,2.0


In [ ]:
# Apply the extraction function to all titles
title_feature_df = df["title"].apply(extract_title_features).apply(pd.Series)

# Merge with the original DataFrame
df = pd.concat([df, title_feature_df], axis=1)
df.head()

,kind,etag,id,channel_id_x,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,starts_with_keyword,title_length,word_count,punctuation_count,uppercase_word_count,percent_letters_uppercase,has_numbers,clickbait_score,has_power_words,has_timed_words
0,youtube#video,UxEp6XJfLMB4Fz5eWVq36fPYfTc,HbyW-LwzXuM,UC0lG3Ihe4LGV851lODRIS5g,2025-06-20T13:45:02Z,UC0lG3Ihe4LGV851lODRIS5g,A one-pot sautéed chicken dish full of old-sch...,A great way to practice your sauté skills with...,https://i.ytimg.com/vi/HbyW-LwzXuM/default.jpg,120,...,0.0,67.0,11.0,2.0,1.0,0.019,0.0,0.0,0.0,0.0
1,youtube#video,DZqEe-E-5XcWRJEjZZj-1v021XE,y9RfNX5RRx8,UC0lG3Ihe4LGV851lODRIS5g,2025-06-09T08:25:09Z,UC0lG3Ihe4LGV851lODRIS5g,How to Learn French Techniques and Recipes,Master the French sauté technique with this si...,https://i.ytimg.com/vi/y9RfNX5RRx8/default.jpg,120,...,1.0,42.0,7.0,0.0,0.0,0.139,0.0,0.0,0.0,0.0
2,youtube#video,Ty1cwmDXz55VVEUXhtPpWAvYvmQ,5y8P_ff-ZR0,UC0lG3Ihe4LGV851lODRIS5g,2025-06-04T10:24:47Z,UC0lG3Ihe4LGV851lODRIS5g,The One Book Every Serious Cook Should Own,"If you’re serious about cooking, the Larousse ...",https://i.ytimg.com/vi/5y8P_ff-ZR0/default.jpg,120,...,0.0,42.0,8.0,0.0,0.0,0.229,0.0,0.0,0.0,0.0
3,youtube#video,Ji2fd_6HSZrZKLubCJeH6Z5Lk7E,1rMmAp2RZhg,UC0lG3Ihe4LGV851lODRIS5g,2025-05-28T10:39:28Z,UC0lG3Ihe4LGV851lODRIS5g,Salted Caramel Ice Cream – A Taste of Our New ...,Our latest course has arrived—just in time for...,https://i.ytimg.com/vi/1rMmAp2RZhg/default.jpg,120,...,0.0,52.0,11.0,0.0,1.0,0.220,0.0,0.0,1.0,0.0
4,youtube#video,cmA8BDkG4mY6BmkpcVrcayicejw,RpBn_deP_ro,UC0lG3Ihe4LGV851lODRIS5g,2025-05-13T11:16:53Z,UC0lG3Ihe4LGV851lODRIS5g,Learn to make French style ice creams 😉,Available now 🍧 Get the Early bird price disco...,https://i.ytimg.com/vi/RpBn_deP_ro/default.jpg,120,...,0.0,39.0,8.0,0.0,0.0,0.065,0.0,0.0,0.0,0.0


In [ ]:
# Export final dataset with image + text features
df.to_csv("video_details_v5.csv", index=False)

# Step 3: Labeling 'virality' using target variables

In [ ]:
# Load the CSV
import pandas as pd

url = "https://raw.githubusercontent.com/rithikkulkarni/Video-Virality/refs/heads/main/data/video_details_v5.csv"
df = pd.read_csv(url)
df.head()

,kind,etag,id,channel_id_x,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,starts_with_keyword,title_length,word_count,punctuation_count,uppercase_word_count,percent_letters_uppercase,has_numbers,clickbait_score,has_power_words,has_timed_words
0,youtube#video,UxEp6XJfLMB4Fz5eWVq36fPYfTc,HbyW-LwzXuM,UC0lG3Ihe4LGV851lODRIS5g,2025-06-20T13:45:02Z,UC0lG3Ihe4LGV851lODRIS5g,A one-pot sautéed chicken dish full of old-sch...,A great way to practice your sauté skills with...,https://i.ytimg.com/vi/HbyW-LwzXuM/default.jpg,120,...,0.0,67.0,11.0,2.0,1.0,0.019,0.0,0.0,0.0,0.0
1,youtube#video,DZqEe-E-5XcWRJEjZZj-1v021XE,y9RfNX5RRx8,UC0lG3Ihe4LGV851lODRIS5g,2025-06-09T08:25:09Z,UC0lG3Ihe4LGV851lODRIS5g,How to Learn French Techniques and Recipes,Master the French sauté technique with this si...,https://i.ytimg.com/vi/y9RfNX5RRx8/default.jpg,120,...,1.0,42.0,7.0,0.0,0.0,0.139,0.0,0.0,0.0,0.0
2,youtube#video,Ty1cwmDXz55VVEUXhtPpWAvYvmQ,5y8P_ff-ZR0,UC0lG3Ihe4LGV851lODRIS5g,2025-06-04T10:24:47Z,UC0lG3Ihe4LGV851lODRIS5g,The One Book Every Serious Cook Should Own,"If you’re serious about cooking, the Larousse ...",https://i.ytimg.com/vi/5y8P_ff-ZR0/default.jpg,120,...,0.0,42.0,8.0,0.0,0.0,0.229,0.0,0.0,0.0,0.0
3,youtube#video,Ji2fd_6HSZrZKLubCJeH6Z5Lk7E,1rMmAp2RZhg,UC0lG3Ihe4LGV851lODRIS5g,2025-05-28T10:39:28Z,UC0lG3Ihe4LGV851lODRIS5g,Salted Caramel Ice Cream – A Taste of Our New ...,Our latest course has arrived—just in time for...,https://i.ytimg.com/vi/1rMmAp2RZhg/default.jpg,120,...,0.0,52.0,11.0,0.0,1.0,0.220,0.0,0.0,1.0,0.0
4,youtube#video,cmA8BDkG4mY6BmkpcVrcayicejw,RpBn_deP_ro,UC0lG3Ihe4LGV851lODRIS5g,2025-05-13T11:16:53Z,UC0lG3Ihe4LGV851lODRIS5g,Learn to make French style ice creams 😉,Available now 🍧 Get the Early bird price disco...,https://i.ytimg.com/vi/RpBn_deP_ro/default.jpg,120,...,0.0,39.0,8.0,0.0,0.0,0.065,0.0,0.0,0.0,0.0


In [ ]:
# Ensure proper numeric types
df["viewCount"] = pd.to_numeric(df["viewCount"], errors="coerce")
df["likeCount"] = pd.to_numeric(df["likeCount"], errors="coerce")
df["commentCount"] = pd.to_numeric(df["commentCount"], errors="coerce")

# Optional: drop rows with missing values
df = df.dropna(subset=["viewCount", "likeCount", "commentCount"])

In [ ]:
new_column_names = {'snippet.channelId': 'channel_id'}
df = df.rename(columns=new_column_names)

In [ ]:
percentile = 0.85

# Select the numeric columns *before* grouping and applying quantile
thresholds = df[["channel_id", "viewCount", "likeCount", "commentCount"]].groupby("channel_id").quantile(percentile)[
    ["viewCount", "likeCount", "commentCount"]
].rename(columns={
    "viewCount": "views_threshold",
    "likeCount": "likes_threshold",
    "commentCount": "comments_threshold"
}).reset_index()

In [ ]:
# Merge thresholds into main dataframe (only run once)
df = df.merge(thresholds, on="channel_id", how="left")
df.head()

,kind,etag,id,channel_id_x,snippet.publishedAt,channel_id,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,punctuation_count,uppercase_word_count,percent_letters_uppercase,has_numbers,clickbait_score,has_power_words,has_timed_words,views_threshold,likes_threshold,comments_threshold
0,youtube#video,UxEp6XJfLMB4Fz5eWVq36fPYfTc,HbyW-LwzXuM,UC0lG3Ihe4LGV851lODRIS5g,2025-06-20T13:45:02Z,UC0lG3Ihe4LGV851lODRIS5g,A one-pot sautéed chicken dish full of old-sch...,A great way to practice your sauté skills with...,https://i.ytimg.com/vi/HbyW-LwzXuM/default.jpg,120,...,2.0,1.0,0.019,0.0,0.0,0.0,0.0,199984.8,5621.0,287.6
1,youtube#video,DZqEe-E-5XcWRJEjZZj-1v021XE,y9RfNX5RRx8,UC0lG3Ihe4LGV851lODRIS5g,2025-06-09T08:25:09Z,UC0lG3Ihe4LGV851lODRIS5g,How to Learn French Techniques and Recipes,Master the French sauté technique with this si...,https://i.ytimg.com/vi/y9RfNX5RRx8/default.jpg,120,...,0.0,0.0,0.139,0.0,0.0,0.0,0.0,199984.8,5621.0,287.6
2,youtube#video,Ty1cwmDXz55VVEUXhtPpWAvYvmQ,5y8P_ff-ZR0,UC0lG3Ihe4LGV851lODRIS5g,2025-06-04T10:24:47Z,UC0lG3Ihe4LGV851lODRIS5g,The One Book Every Serious Cook Should Own,"If you’re serious about cooking, the Larousse ...",https://i.ytimg.com/vi/5y8P_ff-ZR0/default.jpg,120,...,0.0,0.0,0.229,0.0,0.0,0.0,0.0,199984.8,5621.0,287.6
3,youtube#video,Ji2fd_6HSZrZKLubCJeH6Z5Lk7E,1rMmAp2RZhg,UC0lG3Ihe4LGV851lODRIS5g,2025-05-28T10:39:28Z,UC0lG3Ihe4LGV851lODRIS5g,Salted Caramel Ice Cream – A Taste of Our New ...,Our latest course has arrived—just in time for...,https://i.ytimg.com/vi/1rMmAp2RZhg/default.jpg,120,...,0.0,1.0,0.220,0.0,0.0,1.0,0.0,199984.8,5621.0,287.6
4,youtube#video,cmA8BDkG4mY6BmkpcVrcayicejw,RpBn_deP_ro,UC0lG3Ihe4LGV851lODRIS5g,2025-05-13T11:16:53Z,UC0lG3Ihe4LGV851lODRIS5g,Learn to make French style ice creams 😉,Available now 🍧 Get the Early bird price disco...,https://i.ytimg.com/vi/RpBn_deP_ro/default.jpg,120,...,0.0,0.0,0.065,0.0,0.0,0.0,0.0,199984.8,5621.0,287.6


In [ ]:
# Label virality using thresholds
df["viral"] = (
    (df["viewCount"] > df["views_threshold"]) &
    (df["likeCount"] > df["likes_threshold"]) &
    (df["commentCount"] > df["comments_threshold"])
).astype(int)

In [ ]:
# Drop the threshold columns from the main DataFrame
df = df.drop(["views_threshold", "likes_threshold", "comments_threshold"], axis=1)

df.head()

,kind,etag,id,channel_id_x,snippet.publishedAt,channel_id,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,title_length,word_count,punctuation_count,uppercase_word_count,percent_letters_uppercase,has_numbers,clickbait_score,has_power_words,has_timed_words,viral
0,youtube#video,UxEp6XJfLMB4Fz5eWVq36fPYfTc,HbyW-LwzXuM,UC0lG3Ihe4LGV851lODRIS5g,2025-06-20T13:45:02Z,UC0lG3Ihe4LGV851lODRIS5g,A one-pot sautéed chicken dish full of old-sch...,A great way to practice your sauté skills with...,https://i.ytimg.com/vi/HbyW-LwzXuM/default.jpg,120,...,67.0,11.0,2.0,1.0,0.019,0.0,0.0,0.0,0.0,0
1,youtube#video,DZqEe-E-5XcWRJEjZZj-1v021XE,y9RfNX5RRx8,UC0lG3Ihe4LGV851lODRIS5g,2025-06-09T08:25:09Z,UC0lG3Ihe4LGV851lODRIS5g,How to Learn French Techniques and Recipes,Master the French sauté technique with this si...,https://i.ytimg.com/vi/y9RfNX5RRx8/default.jpg,120,...,42.0,7.0,0.0,0.0,0.139,0.0,0.0,0.0,0.0,0
2,youtube#video,Ty1cwmDXz55VVEUXhtPpWAvYvmQ,5y8P_ff-ZR0,UC0lG3Ihe4LGV851lODRIS5g,2025-06-04T10:24:47Z,UC0lG3Ihe4LGV851lODRIS5g,The One Book Every Serious Cook Should Own,"If you’re serious about cooking, the Larousse ...",https://i.ytimg.com/vi/5y8P_ff-ZR0/default.jpg,120,...,42.0,8.0,0.0,0.0,0.229,0.0,0.0,0.0,0.0,0
3,youtube#video,Ji2fd_6HSZrZKLubCJeH6Z5Lk7E,1rMmAp2RZhg,UC0lG3Ihe4LGV851lODRIS5g,2025-05-28T10:39:28Z,UC0lG3Ihe4LGV851lODRIS5g,Salted Caramel Ice Cream – A Taste of Our New ...,Our latest course has arrived—just in time for...,https://i.ytimg.com/vi/1rMmAp2RZhg/default.jpg,120,...,52.0,11.0,0.0,1.0,0.220,0.0,0.0,1.0,0.0,0
4,youtube#video,cmA8BDkG4mY6BmkpcVrcayicejw,RpBn_deP_ro,UC0lG3Ihe4LGV851lODRIS5g,2025-05-13T11:16:53Z,UC0lG3Ihe4LGV851lODRIS5g,Learn to make French style ice creams 😉,Available now 🍧 Get the Early bird price disco...,https://i.ytimg.com/vi/RpBn_deP_ro/default.jpg,120,...,39.0,8.0,0.0,0.0,0.065,0.0,0.0,0.0,0.0,0


In [ ]:
# Calculate the percentage of viral videos
viral_percentage = (df["viral"].sum() / len(df)) * 100

print(f"Percentage of viral videos: {viral_percentage:.2f}%")

Percentage of viral videos: 8.20%


In [ ]:
# Export dataset with image + text features
df.to_csv("video_details_v6.csv", index=False)

# Step 4: Fixing up existing features

In [ ]:
# Load the CSV
import pandas as pd

url = "https://raw.githubusercontent.com/rithikkulkarni/Video-Virality/refs/heads/main/data/video_details_v6.csv"
df = pd.read_csv(url)
df.head()

,kind,etag,id,channel_id_x,snippet.publishedAt,channel_id,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,title_length,word_count,punctuation_count,uppercase_word_count,percent_letters_uppercase,has_numbers,clickbait_score,has_power_words,has_timed_words,viral
0,youtube#video,UxEp6XJfLMB4Fz5eWVq36fPYfTc,HbyW-LwzXuM,UC0lG3Ihe4LGV851lODRIS5g,2025-06-20T13:45:02Z,UC0lG3Ihe4LGV851lODRIS5g,A one-pot sautéed chicken dish full of old-sch...,A great way to practice your sauté skills with...,https://i.ytimg.com/vi/HbyW-LwzXuM/default.jpg,120,...,67.0,11.0,2.0,1.0,0.019,0.0,0.0,0.0,0.0,0
1,youtube#video,DZqEe-E-5XcWRJEjZZj-1v021XE,y9RfNX5RRx8,UC0lG3Ihe4LGV851lODRIS5g,2025-06-09T08:25:09Z,UC0lG3Ihe4LGV851lODRIS5g,How to Learn French Techniques and Recipes,Master the French sauté technique with this si...,https://i.ytimg.com/vi/y9RfNX5RRx8/default.jpg,120,...,42.0,7.0,0.0,0.0,0.139,0.0,0.0,0.0,0.0,0
2,youtube#video,Ty1cwmDXz55VVEUXhtPpWAvYvmQ,5y8P_ff-ZR0,UC0lG3Ihe4LGV851lODRIS5g,2025-06-04T10:24:47Z,UC0lG3Ihe4LGV851lODRIS5g,The One Book Every Serious Cook Should Own,"If you’re serious about cooking, the Larousse ...",https://i.ytimg.com/vi/5y8P_ff-ZR0/default.jpg,120,...,42.0,8.0,0.0,0.0,0.229,0.0,0.0,0.0,0.0,0
3,youtube#video,Ji2fd_6HSZrZKLubCJeH6Z5Lk7E,1rMmAp2RZhg,UC0lG3Ihe4LGV851lODRIS5g,2025-05-28T10:39:28Z,UC0lG3Ihe4LGV851lODRIS5g,Salted Caramel Ice Cream – A Taste of Our New ...,Our latest course has arrived—just in time for...,https://i.ytimg.com/vi/1rMmAp2RZhg/default.jpg,120,...,52.0,11.0,0.0,1.0,0.220,0.0,0.0,1.0,0.0,0
4,youtube#video,cmA8BDkG4mY6BmkpcVrcayicejw,RpBn_deP_ro,UC0lG3Ihe4LGV851lODRIS5g,2025-05-13T11:16:53Z,UC0lG3Ihe4LGV851lODRIS5g,Learn to make French style ice creams 😉,Available now 🍧 Get the Early bird price disco...,https://i.ytimg.com/vi/RpBn_deP_ro/default.jpg,120,...,39.0,8.0,0.0,0.0,0.065,0.0,0.0,0.0,0.0,0


In [ ]:
# Dropping bad columns/features
columns_to_drop = ['kind',
                   'etag',
                   'channel_id_x',
                   'snippet.publishedAt',
                   'snippet.title',
                   'snippet.thumbnails.default.url',
                   'snippet.thumbnails.default.width',
                   'snippet.thumbnails.default.height',
                   'snippet.thumbnails.medium.url',
                   'snippet.thumbnails.medium.width',
                   'snippet.thumbnails.medium.height',
                   'snippet.thumbnails.high.url',
                   'snippet.thumbnails.high.width',
                   'snippet.thumbnails.high.height',
                   'snippet.thumbnails.standard.url',
                   'snippet.thumbnails.standard.width',
                   'snippet.thumbnails.standard.height',
                   'snippet.thumbnails.maxres.url',
                   'snippet.thumbnails.maxres.width',
                   'snippet.thumbnails.maxres.height',
                   'statistics.viewCount',
                   'statistics.likeCount',
                   'statistics.commentCount',
                   'snippet.channelTitle',
                   'snippet.categoryId',
                   'snippet.liveBroadcastContent',
                   'snippet.defaultAudioLanguage',
                   'snippet.defaultLanguage',
                   'title',
                   'channel_id_x',
                   'channel_id_y',
                   'snippet.localized.description',
                   'statistics.favoriteCount',
                   'id']
df = df.drop(columns=columns_to_drop, axis=1)

In [ ]:
df = df.rename(columns={'snippet.description': 'description', 'snippet.localized.title': 'title', 'statistics.viewCount': 'viewCount', 'statistics.likeCount': 'likeCount', 'statistics.commentCount': 'commentCount', 'snippet.tags': 'tags'})

In [ ]:
print(df.columns)

Index(['channel_id', 'description', 'tags', 'title', 'avg_red', 'avg_green',
       'avg_blue', 'brightness', 'contrast', 'video_id', 'viewCount',
       'likeCount', 'commentCount', 'title_sentiment', 'title_subjectivity',
       'has_question', 'has_exclamation', 'starts_with_keyword',
       'title_length', 'word_count', 'punctuation_count',
       'uppercase_word_count', 'percent_letters_uppercase', 'has_numbers',
       'clickbait_score', 'has_power_words', 'has_timed_words', 'viral'],
      dtype='object')


In [ ]:
# Export dataset with fixed up existing features
df.to_csv("video_details_v7.csv", index=False)

# Step 5: Adding new features

In [ ]:
# Load the CSV
import pandas as pd

url = "https://raw.githubusercontent.com/rithikkulkarni/Video-Virality/refs/heads/main/data/video_details_v7.csv"
df = pd.read_csv(url)
df.head()
print(df.columns)

Index(['channel_id', 'description', 'tags', 'title', 'avg_red', 'avg_green',
       'avg_blue', 'brightness', 'contrast', 'video_id', 'viewCount',
       'likeCount', 'commentCount', 'title_sentiment', 'title_subjectivity',
       'has_question', 'has_exclamation', 'starts_with_keyword',
       'title_length', 'word_count', 'punctuation_count',
       'uppercase_word_count', 'percent_letters_uppercase', 'has_numbers',
       'clickbait_score', 'has_power_words', 'has_timed_words', 'viral'],
      dtype='object')


In [ ]:
### Description-based features

import re
from textblob import TextBlob

# Fill NA for descriptions
df['description'] = df['description'].fillna('')

# Description length
df['description_length'] = df['description'].apply(len)

# Description sentiment
df['description_sentiment'] = df['description'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Description has keywords
keywords = ['subscribe', 'giveaway', 'limited time', 'offer', 'new video']
df['description_has_keywords'] = df['description'].apply(lambda x: any(kw in x.lower() for kw in keywords))

In [ ]:
### Tag-based features

# Fill NA for tags
df['tags'] = df['tags'].fillna('').astype(str)

# Tag count
df['tag_count'] = df['tags'].apply(lambda x: len(x.split(',')) if x else 0)

# Tag sentiment
df['tag_sentiment'] = df['tags'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Unique tags
df['num_unique_tags'] = df['tags'].apply(lambda x: len(set(tag.strip().lower() for tag in x.split(','))) if x else 0)

In [ ]:
!pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 59.0 MB/s eta 0:00:00


In [ ]:
### Title-based Semantic features

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import textstat

model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute title embeddings
df['title_embedding'] = df['title'].fillna('').apply(lambda x: model.encode(x, convert_to_tensor=False))

# Embedding distance to known viral videos
viral_embeddings = df[df['viral'] == 1]['title_embedding'].tolist()

def max_cosine_similarity(vec, others):
    return np.max(cosine_similarity([vec], others)[0]) if others else 0

df['embedding_distance_to_known_viral'] = df['title_embedding'].apply(
    lambda x: max_cosine_similarity(x, viral_embeddings)
)

# Clickbait phrase match
clickbait_phrases = ['you won’t believe', 'what happens next', 'this is why', 'top secret', 'never seen before']
df['clickbait_phrase_match'] = df['title'].fillna('').str.lower().apply(lambda t: any(p in t for p in clickbait_phrases))

# Readability score
df['title_readability'] = df['title'].fillna('').apply(lambda x: textstat.flesch_reading_ease(x))

# Embedding distance to average title (unsupervised)
all_embeddings = df['title_embedding'].tolist()
avg_embedding = np.mean(all_embeddings, axis=0)
df['title_embedding_distance_to_viral'] = df['title_embedding'].apply(
    lambda x: 1 - cosine_similarity([x], [avg_embedding])[0][0]
)

# Listicle and tutorial flags
df['is_listicle'] = df['title'].str.strip().str.lower().str.match(r'^\d+').astype(int)
df['is_tutorial'] = df['title'].str.lower().str.startswith('how to').astype(int)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
### Thumbnail-based features
import cv2
import os
from collections import Counter
from matplotlib.colors import rgb_to_hsv

# Path to thumbnail images, assumes filename = <video_id>.jpg
thumbnail_dir = "/content/thumbnails/"

# Face detection setup
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def get_thumbnail_features(video_id):
    path = os.path.join(thumbnail_dir, f"{video_id}.jpg")
    if not os.path.exists(path):
        return pd.Series([0, -1, 0.0], index=['num_faces', 'dominant_color_hue', 'thumbnail_edge_density'])

    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Face count
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    num_faces = len(faces)

    # Dominant color hue
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hue_channel = hsv[:, :, 0].flatten()
    most_common_hue = Counter(hue_channel).most_common(1)[0][0]

    # Edge density
    edges = cv2.Canny(gray, 100, 200)
    edge_density = np.sum(edges > 0) / edges.size

    return pd.Series([num_faces, most_common_hue, edge_density], index=['num_faces', 'dominant_color_hue', 'thumbnail_edge_density'])

df[['num_faces', 'dominant_color_hue', 'thumbnail_edge_density']] = df['video_id'].apply(get_thumbnail_features)

In [ ]:
### Word Count Refinement

# Power/timed word lists
power_words = ['secret', 'shocking', 'revealed', 'proven', 'insane']
timed_words = ['today', 'now', 'soon', 'in 24 hours', 'this week']

def count_matches(text, word_list):
    return sum(word in text.lower() for word in word_list)

df['power_word_count'] = df['title'].fillna('').apply(lambda x: count_matches(x, power_words))
df['timed_word_count'] = df['title'].fillna('').apply(lambda x: count_matches(x, timed_words))

# Optionally drop the binary flags
df.drop(columns=['has_power_words', 'has_timed_words'], inplace=True, errors='ignore')

In [ ]:
# Checking through added features

print(df.columns)
print(df.head)
timed_word_count_sum = df['timed_word_count'].sum()
power_word_count_sum = df['power_word_count'].sum()
is_listicle_sum = df['is_listicle'].sum()
is_tutorial_sum = df['is_tutorial'].sum()
num_faces_sum = df['num_faces'].sum()
true_clickbait_matches = df['clickbait_phrase_match'].sum()
print(f"Total number of timed_word_count values: {timed_word_count_sum}")
print(f"Total number of power_word_count values: {power_word_count_sum}")
print(f"Total number of is_listicle values: {is_listicle_sum}")
print(f"Total number of is_tutorial values: {is_tutorial_sum}")
print(f"Total number of num_faces values: {num_faces_sum}")
print(f"Total number of true clickbait_phrase_match values: {true_clickbait_matches}")

Index(['channel_id', 'description', 'tags', 'title', 'avg_red', 'avg_green',
       'avg_blue', 'brightness', 'contrast', 'video_id', 'viewCount',
       'likeCount', 'commentCount', 'title_sentiment', 'title_subjectivity',
       'has_question', 'has_exclamation', 'starts_with_keyword',
       'title_length', 'word_count', 'punctuation_count',
       'uppercase_word_count', 'percent_letters_uppercase', 'has_numbers',
       'clickbait_score', 'viral', 'description_length',
       'description_sentiment', 'description_has_keywords', 'tag_count',
       'tag_sentiment', 'num_unique_tags', 'title_embedding',
       'embedding_distance_to_known_viral', 'clickbait_phrase_match',
       'title_readability', 'title_embedding_distance_to_viral', 'is_listicle',
       'is_tutorial', 'num_faces', 'dominant_color_hue',
       'thumbnail_edge_density', 'power_word_count', 'timed_word_count'],
      dtype='object')
<bound method NDFrame.head of                      channel_id  \
0      UC0lG3Ihe4L

In [ ]:
# Removing any seemingly useless features
columns_to_drop = ['timed_word_count', 'power_word_count', 'is_listicle', 'is_tutorial', 'num_faces', 'clickbait_phrase_match']
df = df.drop(columns=columns_to_drop, axis=1)

In [ ]:
df['log_viewCount'] = np.log1p(df['viewCount'])
df['log_likeCount'] = np.log1p(df['likeCount'])
df['log_commentCount'] = np.log1p(df['commentCount'])

In [ ]:
# Export
df.to_csv("video_details_v8.csv", index=False)